<a href="https://colab.research.google.com/github/Friedrichz/crypto_tooling/blob/main/track_vc_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install messari

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from messari.messari import Messari
import pandas as pd

messari_api_key = "88bbfa31-1b0c-442b-b748-cc13ed2cab1a"
messari = Messari(messari_api_key)

In [4]:
li_dic = [] 
for i in range(1, 5, 1):
    data = messari.get_all_assets(page=i, limit=500, to_dataframe=True)
    li_dic.append(data)

In [5]:
data = li_dic[0]['data']
len(data)

500

In [67]:
record = data[13]
# Extract (current) market data --> metrics: token_sale_stats>sale_proceeds_usd, supply_distribution
record["metrics"].keys()#["sale_proceeds_usd"]

dict_keys(['market_data', 'marketcap', 'supply', 'blockchain_stats_24_hours', 'market_data_liquidity', 'all_time_high', 'cycle_low', 'token_sale_stats', 'mining_stats', 'developer_activity', 'roi_data', 'roi_by_year', 'risk_metrics', 'misc_data', 'reddit', 'on_chain_data', 'exchange_flows', 'miner_flows', 'supply_activity', 'supply_distribution', 'alert_messages'])

In [61]:
# Extract initial private fundraising data --> profile: investors>organizations, economics>launch>fundraising

def get_nested_items(list_records, attr): 
  if list_records is None:
    return []
  else: 
    return [i[attr] for i in list_records]

def clean_sales_round_title(title): 
  if title is None: pass
  else:
    # Remove digits
    title = ''.join([i for i in title if not i.isdigit()]) 
    # Lowercase & strip trailing/leading spaces
    title = title.lower().strip()
    return title

def clean_sales_rounds(list_records): 
  if list_records is None:
    pass
  else:
    [i.update(title_clean=clean_sales_round_title(i["title"])) for i in list_records]
    return

# Get list of sales_rounds labels to identify venture round / seed sale 
sales_rounds_titles = set(j for i in data for j in get_nested_items(i["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"], "title"))

# Taken manually from list of titles in data 
early_sales_rounds = [
  'Fundraising ', 
  'Pre Seed Round', 
  'Pre-Sale SAFT', 
  'Private Presale', 
  'Private Sale', 
  'Private Sale1 ',
  'SAFT',
  'Seed', 
  'Seed ', 
  'Seed Funding', 
  'Seed Round', 
  'Seed Round (SAFT) ', 
  'Seed Sale', 
  'Seed Round (Crypto)', 
  'Seed Token Sale',
  'Strategic Funding Round',
  'Strategic Investments',
  'Strategic Private Sale',
  'Strategic Round',
  'Strategic Sale',
  'Token Pre-Sale',
  'Token Private Round',
  'Token Seed Round',
  'Venture Capital',
  'Venture Funding',
  'Venture Round',
]

# Clean up sales round titles & add "title_clean" to records
early_sales_rounds_clean = set(clean_sales_round_title(i) for i in early_sales_rounds)
print(len(early_sales_rounds_clean)) # Include 24 titles that point to private token sales
_ = [clean_sales_rounds(j["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]) for j in data]

# Get list of private investors
investors = [i["name"] for i in record["profile"]["investors"]["organizations"]]

# Filter records of private token sales
private_rounds = [i for i in record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"] if i["title_clean"] in early_sales_rounds_clean]

# Get earliest & last date of private sales
first_private_sale_date = min([i["end_date"] for i in private_rounds])
last_private_sale_date = max([i["end_date"] for i in private_rounds])

# Get total number of tokens sold during private sales
num_tokens_private_sale = sum([i["native_tokens_allocated"] for i in private_rounds])

# Get total USD amount raised in private sales
tot_usd_raised_private_sale = sum([i["amount_collected_in_usd"] for i in private_rounds])

# Get last private sale token price 
min_price_private_sale = min([i["equivalent_price_per_token_in_usd"] for i in private_rounds])
max_price_private_sale = max([i["equivalent_price_per_token_in_usd"] for i in private_rounds])

# Get weighted avg token price of private sales
wavg_price_private_sale = sum([i["amount_collected_in_usd"]*i["equivalent_price_per_token_in_usd"] for i in private_rounds]) / tot_usd_raised_private_sale

# Get total token supply
tot_supply = record["profile"]["economics"]["launch"]["initial_distribution"]["initial_supply"]

# Calc FDV based on last private sale token price
fdv_private_sale = max_price_private_sale * tot_supply

# Safety check: currency collected in private fundraise
asset_collected = set(i["asset_collected"] for i in private_rounds)




24


In [62]:
print(first_private_sale_date, last_private_sale_date, num_tokens_private_sale, tot_usd_raised_private_sale, min_price_private_sale, max_price_private_sale, wavg_price_private_sale, tot_supply, fdv_private_sale)

2019-02-01T05:00:00Z 2019-02-01T05:00:00Z 18000000 5940000 0.33 0.33 0.33 360000000 118800000.0


In [63]:
360000000* 0.33

118800000.0

In [46]:
data[24]["name"]
# [i["name"] for i in data[24]["profile"]["investors"]["organizations"]]
[i for i in data[24]["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]]

[{'amount_collected_in_asset': None,
  'amount_collected_in_usd': 810000,
  'asset_collected': 'USD',
  'details': 'This is the first private sale that NEAR protocol conducted in Q3 2017. NEAR protocol offered 21.6 million NEAR at the rate of $0.0375 per token, raising $810,000.\n\nNEAR [announced](https://www.businesswire.com/news/home/20190710005723/en/NEAR-Protocol-Developers-Raise-12.1M-Round-led-by-Metastable-and-Accomplice-to-Bridge-the-Adoption-Gap-for-Decentralized-Applications) that this raise was a part of its initial venture round, which brought in over $12 million. The team disclosed these details in July 2019.',
  'end_date': '2017-09-30T00:00:00Z',
  'equivalent_price_per_token_in_usd': 0.0375,
  'is_kyc_required': None,
  'native_tokens_allocated': 21600000,
  'price_per_token_in_asset': None,
  'restricted_jurisdictions': None,
  'start_date': '2017-06-01T00:00:00Z',
  'title': 'Private Sale1 ',
  'title_clean': 'private sale',
  'type': 'Token Sale'},
 {'amount_collect

In [32]:
(2500000*0.15+70666666*0.12+ 57600000*0.05+21600000*0.0375) / (2500000+70666666+57600000+21600000)

0.08233428117407254

In [ ]:
url = "https://data.messari.io/api/v2/assets?with-profiles"
page = 2
assets_bytes = urllib.request.urlopen(url + "?page={}".format(str(page))).read()
assets_dict = json.loads(assets_bytes.decode('utf-8'))

In [ ]:
len(assets_dict)
len(assets_dict["data"])

2